In [1]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from energy_cycle import energy_generating_cycle_test

# read model for E. coli K12
k12 = read_sbml_model('../GEMs/iML1515.xml')

#read the universal model
uni = read_sbml_model('../GEMs/bigg_universe.xml')

No objective coefficients in model. Unclear what should be optimized


# CAL2 module

In [12]:
CAL2 = k12.copy()

## Model assembly
### Knock-outs

In [13]:
from cobra.manipulation import knock_out_model_genes

# pheA | gene b2599
knock_out_model_genes(CAL2, ["b2599"])

[<Reaction PPNDH at 0x132b066e0>]

### Metabolites

In [25]:
#p-Coumaric acid
p_coumaric_acid_c = uni.metabolites.get_by_id("T4hcinnm_c")

#caffeic acid
caffeic_acid_c = uni.metabolites.get_by_id("34dhcinm_c")

#external caffeic acid
caffeic_acid_e = uni.metabolites.get_by_id("34dhcinm_e")

#ammonia
nh4_c = k12.metabolites.get_by_id("nh4_c")

#tyrosine 
tyr__L_c = k12.metabolites.get_by_id("tyr__L_c")

#NADH
nadh_c = k12.metabolites.get_by_id("nadh_c")

#NAD+
nad_c = k12.metabolites.get_by_id("nad_c")

#H+
h_c = k12.metabolites.get_by_id("h_c")

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
adp_c = k12.metabolites.get_by_id("adp_c")

#Diphosphate
pi_c = k12.metabolites.get_by_id("pi_c")

### Reactions

In [14]:
#tyr -> p-coumaric acid
TAL = Reaction('TAL')
TAL.name = 'L-tyrosine ammonia-lyase'
TAL.subsystem = 'CA module'
TAL.lower_bound = -1000  # This is the default
TAL.upper_bound = 1000  # This is the default
#L-Tyrosine <=> 4-Coumarate + Ammonia
TAL.add_metabolites({
    tyr__L_c: -1.0,
    p_coumaric_acid_c: 1.0,
    nh4_c: 1.0
})
#TAL.gene_reaction_rule = 'tal' #synthetic rgTAL gene

CAL2.add_reactions([TAL])

In [15]:
#p-coumaric acid -> caffeic acid
COURCA = Reaction('COURCA')
COURCA.name = '4-hydroxyphenylacetate 3-hydroxylase'
COURCA.subsystem = 'CA module'
COURCA.lower_bound = -1000  # This is the default
COURCA.upper_bound = 1000  # This is the default
#4-Coumarate <=> Caffeate
COURCA.add_metabolites({
    p_coumaric_acid_c: -1.0,
    nadh_c: -1.0,
    caffeic_acid_c: 1.0,
    nad_c: 1.0,
    h_c: 1.0
})
#COURCA.gene_reaction_rule = '(hpaB and hpaC)' #synthetic hpaC gene

CAL2.add_reactions([COURCA])

In [17]:
#caffeic acid transport reaction
CAFFAt = Reaction('34DHCINMt')
CAFFAt.name = 'caffeic acid active transport'
CAFFAt.subsystem = 'CA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffeic_acid_c: -1.0,
    atp_c: -1.0,
    caffeic_acid_e: 1.0,
    adp_c: 1.0,
    pi_c: 1.0
})

CAL2.add_reactions([CAFFAt])

In [26]:
#add exhange reaction

CAL2.metabolites.get_by_id("34dhcinm_e").compartment = 'e' #set compartment to be 'e' rather than 'C_e'


CAL2.add_boundary(CAL2.metabolites.get_by_id("34dhcinm_e"), type="exchange")

Reaction identifier,EX_34dhcinm_e
Name,"3,4-Dihydroxy-trans-cinnamate exchange"
Memory address,0x133021fc0
Stoichiometry,"34dhcinm_e <=> 3,4-Dihydroxy-trans-cinnamate <=>"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Testing model

In [27]:
#test for energy-generating cycles
energy_generating_cycle_test(CAL2)

ATPM flux: 0.0


,fluxes,reduced_costs
CYTDK2,0.0,-2.000000
XPPT,0.0,10.182815
HXPRT,0.0,10.182815
NDPK5,0.0,-1.908592
SHK3Dr,0.0,2.000000
...,...,...
OCTNLL,0.0,4.182815
TAL,0.0,-0.726552
COURCA,0.0,2.000000
34DHCINMt,0.0,-2.000000


In [29]:
#adding Phe to the medium
phe_medium = CAL2.medium
phe_medium["EX_phe__L_e"] = 1000.0
CAL2.medium = phe_medium

#stopping caffeic acid uptake so that the model does not use this as substrate
CAL2.reactions.EX_34dhcinm_e.lower_bound = 0

In [40]:
from cobra.flux_analysis import flux_variability_analysis

CAL2.reactions.EX_34dhcinm_e.summary(fva=0.95)

## Export model to SBML

In [ ]:
from cobra.io import write_sbml_model

write_sbml_model(CAL2, "../GEMs/CAL2.xml")